# BERT Fine-Tuning 
This notebook contains code for fine-tuning BERT on the TrueFake Dataset. The model fine-tuned corresponds to the Veracity classifier in the Fake News Detection Framework.

The second part of the notebook contains evaluations of the fine-tuned BERT model on various test datasets.

# Data Split
Training 60%
Validation 20%
Test 20%


-- Dataset is based on Title/Text/Label crawled from websources and kaggle data

In [1]:
import pandas as pd
import numpy as np
import csv
import sklearn as sk
from sklearn.metrics import classification_report
from tqdm.auto import tqdm, trange

In [ ]:
train_data_len = 60/100
test_data_len = 40/100

In [ ]:
# Read Entire Dataset
df_original = pd.read_csv("Datasets/ds_true_fake.csv")

In [ ]:
df_original.head()

In [ ]:
df_original = df_original.sample(frac = 1).reset_index(drop=True)
df_original.count()

In [ ]:
c_title = list(df_original['title'].str.lower())
c_text = list(df_original['text'].str.lower())
c_label = list(df_original['label'])
print("Total Dataset length : %d" %(len(c_text)))

In [ ]:
#Split Data to training and test set
t_index = int(train_data_len * len(c_text))
print(t_index)

In [ ]:
#training data
df_train = {}
df_train['Title'] = c_title[:t_index]
df_train['Text'] = c_text[:t_index]
df_train['Label'] = c_label[:t_index] 
print("Training data length: % d" %(len(df_train['Title'])))

In [ ]:
#test data
df_test = {}
df_test['Title'] = c_title[t_index:]
df_test['Text'] = c_text[t_index:]
df_test['Label'] = c_label[t_index:]
print("Test data length: % d" %(len(df_test['Title'])))

In [ ]:
ds_train_true_fake = pd.concat([pd.Series(v, name=k) for k,v in df_train.items()], axis = 1)
ds_test_true_fake = pd.concat([pd.Series(v, name=k) for k,v in df_test.items()], axis = 1)

In [ ]:
ds_train_true_fake.to_csv("Datasets/Data/28Feb_train.csv", index=False)
ds_test_true_fake.to_csv("Datasets/Data/28Feb_test.csv", index=False)

In [ ]:
#save as pkl
ds_train_true_fake.to_pickle("Datasets/Data/28Feb_train.pkl")
ds_test_true_fake.to_pickle("Datasets/Data/28Feb_test.pkl")

# Baseline: Original Dataset
The model is trained on the original dataset, where the sources are scraped online news article and Kaggle dataset

In [2]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
torch.cuda.current_device()

0

In [3]:
# Use this code when on GPU
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=4

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=4


In [4]:
BATCH_SIZE = 16
EPOCHS = 30

In [5]:
torch.cuda.empty_cache()

In [6]:
#Train Data 
train_path = "Datasets/Data/train.csv"
train_df = pd.read_csv(train_path)
train_df.columns = ["Title", "Text", "Label"]
train_df.dropna()
print("Length of Train Set:" + str(len(train_df)))
train_df.head()

Length of Train Set:27716


,Title,Text,Label
0,israel disappointed at trump's decision not to...,israel on thursday expressed disappointment a...,0
1,ugandan mps cite intimidation ahead of move to...,ugandan lawmakers accused the government on w...,0
2,democrat gun grabber sit-in rages on with a da...,democrats staging a sit-in on the floor of con...,1
3,women in asia-pacific express dismay over u.s....,women in the asia-pacific region expressed di...,0
4,ouch! joe biden’s former staffer’s scorching f...,who knew biden is a tyrant? the author of a 20...,1


In [8]:
27716 + 18478

46194

# Model Definition

In [9]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

In [10]:
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [11]:
_train_title = [str(stmt) for stmt in train_df['Title']]
_train_text = [str(jst) for jst in train_df['Text']]
_train_labels = list(train_df['Label'])

In [12]:
print(len(_train_title))
print(len(_train_text))
print(len(_train_labels))

27716
27716
27716


In [13]:
# Data Split 
train_title, val_title, train_text, val_text, train_labels, val_labels = train_test_split(_train_title, _train_text, _train_labels, 
                                                                    random_state=2018, 
                                                                    test_size=0.2, 
                                                                    stratify=_train_labels)

In [14]:
type(val_text[0])

str

In [15]:
title_tokens_train = tokenizer.batch_encode_plus(
    train_title,
    max_length = 100,
    padding=True,
    truncation=True
)

text_tokens_train = tokenizer.batch_encode_plus(
     train_text,
     max_length = 400,
     padding=True,
     truncation=True
)

title_tokens_val = tokenizer.batch_encode_plus(
    val_title,
    max_length = 100,
    padding=True,
    truncation=True
)

text_tokens_val = tokenizer.batch_encode_plus(
     val_text,
     max_length = 400,
     padding=True,
     truncation=True
)

In [16]:
title_seq = torch.tensor(title_tokens_train['input_ids'])
text_seq = torch.tensor(text_tokens_train['input_ids'])
train_seq = torch.cat((title_seq, text_seq), 1)

title_mask = torch.tensor(title_tokens_train['attention_mask'])
text_mask = torch.tensor(text_tokens_train['attention_mask'])
train_mask = torch.cat((title_mask, text_mask), 1)

title_seq_val = torch.tensor(title_tokens_val['input_ids'])
text_seq_val = torch.tensor(text_tokens_val['input_ids'])
val_seq = torch.cat((title_seq_val, text_seq_val), 1)

title_mask_val = torch.tensor(title_tokens_val['attention_mask'])
text_mask_val = torch.tensor(text_tokens_val['attention_mask'])
val_mask = torch.cat((title_mask_val, text_mask_val), 1)

In [17]:
train_y = torch.tensor(train_labels)

In [18]:
val_y = torch.tensor(val_labels)

In [19]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [20]:
train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = BATCH_SIZE)

In [21]:
val_data = TensorDataset(val_seq, val_mask, val_y)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=BATCH_SIZE)

In [22]:
# freeze all the parameters of bert, no finetuning of bert
for param in bert.parameters():
    param.requires_grad = False

In [23]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
        super(BERT_Arch, self).__init__()
        self.bert = bert 
      
        # relu activation function
        self.relu =  nn.ReLU()
        # dense layer 1
        self.fc1 = nn.Linear(768,512)
        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,2)
        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)
        #define the forward pass
    def forward(self, sent_id, mask):
        #pass the inputs to the model  
        _, cls_hs = self.bert(sent_id, attention_mask=mask)
        x = self.fc1(cls_hs)
        x = self.relu(x)
        # x = self.dropout(x)
        # output layer
        x = self.fc2(x)
        # apply softmax activation
        x = self.softmax(x)
        return x

In [24]:
model = BERT_Arch(bert)
device = torch.device('cuda')

# push the model to GPU
model = model.to(device)

In [25]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
lr = 1e-5
optimizer = AdamW(model.parameters(), lr = lr) 

In [26]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_weights = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print("Class Weights:",class_weights)

Class Weights: [1.0361716  0.96626863]


/srv/home/8vijayak/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass classes=[0 1], y=[1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 

In [27]:
# converting list of class weights to a tensor
weights= torch.tensor(class_weights,dtype=torch.float)

# push to GPU
weights = weights.to(device)

# define the loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

In [28]:
#function to train the model
def train(model, train_dataloader, verbose=True):
    model.train()
    total_loss, total_accuracy = 0, 0
    total_preds=[]
    
    y_true = []
    metrics = {}
    step=0
    
       
    for batch in tqdm(train_dataloader, desc="training..."):
        step = step + 1
               
        batch = [r.to(device) for r in batch]
        sent_id, mask, labels = batch
        y_true.append(labels.detach().cpu().numpy())
        
        model.zero_grad()
        preds = model(sent_id, mask)
        loss = cross_entropy(preds, labels)
        total_loss = total_loss + loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        preds=preds.detach().cpu().numpy()
        total_preds.append(preds)
  
    avg_loss = total_loss / len(train_dataloader)
    total_preds  = np.concatenate(total_preds, axis=0)
    y_true  = np.concatenate(y_true, axis=0)
    
      
    
    metrics['loss'] = avg_loss
    metrics['accuracy']  = sk.metrics.accuracy_score(y_true, np.argmax(total_preds, axis = 1))
    metrics['f1'] = sk.metrics.f1_score(y_true, np.argmax(total_preds, axis = 1))
    if verbose:
        print(f'Training Statistics:')
        print(f'loss: {metrics["loss"]}')    
        print(f'accuracy: {metrics["accuracy"]}')   
        print(f'f1: {metrics["f1"]}')    

    return metrics, total_preds, model

In [29]:
# function for evaluating the model
def evaluate(model, val_dataloader, verbose=True):
    print("\nEvaluating...")
    model.eval()
    total_loss, total_accuracy = 0, 0
    total_preds = []
    y_true = []
    metrics = {}
    step = 0
    
    for batch in tqdm(val_dataloader, desc="evaluating..."):
        step=step+1
        batch = [t.to(device) for t in batch]
        sent_id, mask, labels = batch
        y_true.append(labels.detach().cpu().numpy())
        with torch.no_grad():
            preds = model(sent_id, mask)
            loss = cross_entropy(preds,labels)
            total_loss = total_loss + loss.item()
            preds = preds.detach().cpu().numpy()
            total_preds.append(preds)
            
    avg_loss = total_loss / len(val_dataloader) 
    total_preds  = np.concatenate(total_preds, axis=0)
    y_true  = np.concatenate(y_true, axis=0)
    
    metrics['loss'] = avg_loss
    metrics['accuracy']  = sk.metrics.accuracy_score(y_true, np.argmax(total_preds, axis = 1))
    metrics['f1'] = sk.metrics.f1_score(y_true, np.argmax(total_preds, axis = 1))
    
    if verbose:
        print(f'Validation Statistics:')
        print(f'loss: {metrics["loss"]}')    
        print(f'accuracy: {metrics["accuracy"]}')   
        print(f'f1: {metrics["f1"]}')    

    return metrics, total_preds

In [30]:
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

In [31]:
prefix = 'TrainedModels/ds_weights'

fp = open(f"{prefix}_train_progress_ep-{EPOCHS}_lr-{str(lr).replace('-','')}.tsv", 'w')
fp.write(f'epoch\ttrain_loss\ttrain_accuracy\teval_loss\teval_accuracy\n')

56

In [32]:
for epoch in range(EPOCHS):
    print('\n Epoch {:} / {:}'.format(epoch + 1, EPOCHS))
    
    #train model
    #train_loss, _total_preds = train()
    train_metrics, train_preds, model = train(model, train_dataloader)
    
    #evaluate model
    #valid_loss, _total_preds = evaluate()
    valid_metrics, valid_preds = evaluate(model, val_dataloader)

    #save the best model
    if valid_metrics['loss'] < best_valid_loss:
        best_valid_loss = valid_metrics['loss']
        best_metrics = valid_metrics
        torch.save(model.state_dict(), f"{prefix}_{EPOCHS}_lr-{str(lr).replace('-','')}.pt")
    
    # append training and validation loss
    train_losses.append(train_metrics['loss'])
    valid_losses.append(valid_metrics['loss'])
    
    print(f'\ntrain:\n {train_metrics}')
    print(f'\nvalid:\n {valid_metrics}')
    fp.write(f"{epoch+1}\t{train_metrics['loss']}\t{train_metrics['accuracy']}\t{valid_metrics['loss']}\t{valid_metrics['accuracy']}\n")
    
fp.close()  


 Epoch 1 / 30


training...:   0%|          | 0/1386 [00:00<?, ?it/s]

Training Statistics:
loss: 0.5241518811488048
accuracy: 0.8362348908533285
f1: 0.8415171751560386


evaluating...:   0%|          | 0/347 [00:00<?, ?it/s]

Validation Statistics:
loss: 0.3615650455615019
accuracy: 0.895021645021645
f1: 0.8924611973392461

train:
 {'loss': 0.5241518811488048, 'accuracy': 0.8362348908533285, 'f1': 0.8415171751560386}

valid:
 {'loss': 0.3615650455615019, 'accuracy': 0.895021645021645, 'f1': 0.8924611973392461}

 Epoch 2 / 30


...

Validation Statistics:
loss: 0.12393906205605383
accuracy: 0.9574314574314574
f1: 0.958523725834798

train:
 {'loss': 0.14759752446598096, 'accuracy': 0.9498917553671297, 'f1': 0.9513551381408993}

valid:
 {'loss': 0.12393906205605383, 'accuracy': 0.9574314574314574, 'f1': 0.958523725834798}

 Epoch 30 / 30


training...:   0%|          | 0/1386 [00:00<?, ?it/s]

Training Statistics:
loss: 0.1460964480357199
accuracy: 0.9494407360635035
f1: 0.9508957904419817


evaluating...:   0%|          | 0/347 [00:00<?, ?it/s]

Validation Statistics:
loss: 0.12501006135623097
accuracy: 0.9554473304473304
f1: 0.9563681328387211

train:
 {'loss': 0.1460964480357199, 'accuracy': 0.9494407360635035, 'f1': 0.9508957904419817}

valid:
 {'loss': 0.12501006135623097, 'accuracy': 0.9554473304473304, 'f1': 0.9563681328387211}


In [ ]:
path = "TrainedModels/ds_weights_30_lr-1e05.pt"
model.load_state_dict(torch.load(path))

In [ ]:
model.state_dict()

In [33]:
def predict(model, test_dataloader, verbose=False):
    #print("\nPredicting...")
    model.eval()
    total_loss, total_accuracy = 0, 0
    total_preds = []
    y_true = []
    metrics = {}
    step=0
    
    #for step,batch in enumerate(test_dataloader):
        #if step % 50 == 0 and not step == 0:
            #print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(test_dataloader)))
    for batch in tqdm(test_dataloader, desc="predicting..."):
        step=step+1
        batch = [t.to(device) for t in batch]
        sent_id, mask, labels = batch
        y_true.append(labels.detach().cpu().numpy())
        with torch.no_grad():
            preds = model(sent_id, mask)
            loss = cross_entropy(preds,labels)
            total_loss = total_loss + loss.item()
            preds = preds.detach().cpu().numpy()
            total_preds.append(preds)
    avg_loss = total_loss / len(test_dataloader) 
    total_preds  = np.concatenate(total_preds, axis=0).argmax(axis=1)
    y_true  = np.concatenate(y_true, axis=0)
    
    metrics['loss'] = avg_loss
    metrics['accuracy'] = sk.metrics.accuracy_score(y_true, total_preds)
    metrics['f1'] = sk.metrics.f1_score(y_true, total_preds)
    
    return metrics, total_preds

# TrueFake Test Dataset

In [34]:
test_path = "Datasets/Data/test.csv"
test_df = pd.read_csv(test_path)
test_df.columns = ["Title", "Text", "Label"]
test_df.dropna()
print("Length of Test Set:" + str(len(test_df)))
test_df.head()

Length of Test Set:18478


,Title,Text,Label
0,franklin graham’s daughter: 9/11 happened bec...,religious bigots continue their hateful rhetor...,1
1,obama’s brother will vote for trump: “deep dis...,the tsunami has started president obama s keny...,1
2,unreal! two florida immigrant politicians inti...,"commissioner mack bernard and rep. al jacquet,...",1
3,"before the white house, kennedy was a high-sch...",years before he captained the torpedo boat pt...,0
4,india says committed to climate pact despite u...,india remains committed to the paris agreemen...,0


In [35]:
_test_title = [str(stmt) for stmt in test_df['Title']]
_test_text = [str(justification) for justification in test_df['Text']]
_test_labels = list((test_df['Label']))

In [36]:
title_tokens_test = tokenizer.batch_encode_plus(
    _test_title,
    max_length = 100,
    padding=True,
    truncation=True
)

In [37]:
text_tokens_test = tokenizer.batch_encode_plus(
    _test_text,
    max_length = 400,
    padding=True,
    truncation=True
)

In [38]:
title_seq_test = torch.tensor(title_tokens_test['input_ids'])
text_seq_test = torch.tensor(text_tokens_test['input_ids'])
test_seq = torch.cat((title_seq_test, text_seq_test), 1)

In [39]:
title_mask_test = torch.tensor(title_tokens_test['attention_mask'])
text_mask_test = torch.tensor(text_tokens_test['attention_mask'])
test_mask = torch.cat((title_mask_test, text_mask_test), 1)

In [40]:
# test_y = labels_test
test_y = torch.tensor(_test_labels)

In [41]:
test_data = TensorDataset(test_seq, test_mask, test_y)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size=BATCH_SIZE)

In [42]:
score_metrics, t_predictions = predict(model, test_dataloader)

predicting...:   0%|          | 0/1155 [00:00<?, ?it/s]

In [43]:
print("Model: Test Set: Metrics:")
print(score_metrics)

Model: Test Set: Metrics:
{'loss': 0.13544700798614845, 'accuracy': 0.9550276003896525, 'f1': 0.9556870900655895}


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_y, t_predictions))

# LIAR Test Dataset

In [ ]:
#Test Set is the LIAR Dataset
test_path = "Datasets/ds_liar_test.tsv"
test_LIAR_df = pd.read_csv(test_path, sep='\t')
test_LIAR_df.columns = ["Label", "Statement", "Justification"]
test_LIAR_df.dropna()
print("Length of Test Set:" + str(len(test_LIAR_df)))
test_LIAR_df.head()

In [ ]:
liar_text_test = list(test_LIAR_df['Statement'])
#text_test = [str(stmt) for stmt in test_LIAR_df['Statement']]
liar_justification_test = [str(justification) for justification in test_LIAR_df['Justification']]
liar_labels_test = list(test_LIAR_df['Label'])

text_tokens_test1 = tokenizer.batch_encode_plus(
    liar_text_test,
    max_length = 100,
    padding=True,
    truncation=True
)

justification_tokens_test1 = tokenizer.batch_encode_plus(
    liar_justification_test,
    max_length = 400,
    padding=True,
    truncation=True
)

text_seq_test1 = torch.tensor(text_tokens_test1['input_ids'])
justification_seq_test1 = torch.tensor(justification_tokens_test1['input_ids'])
test_seq1 = torch.cat((text_seq_test1, justification_seq_test1), 1)

text_mask_test1 = torch.tensor(text_tokens_test1['attention_mask'])
justification_mask_test1 = torch.tensor(justification_tokens_test1['attention_mask'])
test_mask1 = torch.cat((text_mask_test1, justification_mask_test1), 1)

# test_y = labels_test
test_y1 = torch.tensor(liar_labels_test)

test_data1 = TensorDataset(test_seq1, test_mask1, test_y1)
test_sampler1 = SequentialSampler(test_data1)
test_dataloader1 = DataLoader(test_data1, sampler = test_sampler1, batch_size=BATCH_SIZE)

In [ ]:
score_metrics1, t_predictions1= predict(model, test_dataloader1)

In [ ]:
score_metrics1

# EUvsDisInfo Test Dataset

In [2]:
#Test Set is the pure EUvsDisInfo Dataset
test_eu_path = "Datasets/ds_EUvsDisInfo_test.tsv"
test_EUvsDisInfo_df = pd.read_csv(test_eu_path, sep='\t')
test_EUvsDisInfo_df.columns = ["Statement", "Justification", "Label"]
test_EUvsDisInfo_df.dropna()
print("Length of Test Set:" + str(len(test_EUvsDisInfo_df)))
test_EUvsDisInfo_df.head()

Length of Test Set:4551


,Statement,Justification,Label
0,opcw pressured by us to issue biased report on...,this article is a part of an ongoing disinform...,1
1,european elites deliberately spread unfounded ...,recurring pro-kremlin narrative that allegatio...,1
2,the opcw ignores data confirming the douma che...,the opcw is clear about the methodology [https...,1
3,poroshenko wanted to trade crimea for membersh...,recurring pro-kremlin disinformation narrative...,1
4,crimea's residents decided to rejoin russia th...,recurrent pro-kremlin disinformation narrative...,1


In [ ]:
EUvsDisInfo_text_test = list(test_EUvsDisInfo_df['Statement'])
#text_test = [str(stmt) for stmt in test_LIAR_df['Statement']]
EUvsDisInfo_liar_justification_test = [str(justification) for justification in test_EUvsDisInfo_df['Justification']]
EUvsDisInfo_liar_labels_test = list(test_EUvsDisInfo_df['Label'])

text_tokens_test2 = tokenizer.batch_encode_plus(
    EUvsDisInfo_text_test,
    max_length = 100,
    padding=True,
    truncation=True
)

justification_tokens_test2 = tokenizer.batch_encode_plus(
    EUvsDisInfo_liar_justification_test,
    max_length = 400,
    padding=True,
    truncation=True
)

text_seq_test2 = torch.tensor(text_tokens_test2['input_ids'])
justification_seq_test2 = torch.tensor(justification_tokens_test2['input_ids'])
test_seq2 = torch.cat((text_seq_test2, justification_seq_test2), 1)

text_mask_test2 = torch.tensor(text_tokens_test2['attention_mask'])
justification_mask_test2 = torch.tensor(justification_tokens_test2['attention_mask'])
test_mask2 = torch.cat((text_mask_test2, justification_mask_test2), 1)

# test_y = labels_test
test_y2 = torch.tensor(EUvsDisInfo_liar_labels_test)

test_data2 = TensorDataset(test_seq2, test_mask2, test_y2)
test_sampler2 = SequentialSampler(test_data2)
test_dataloader2 = DataLoader(test_data2, sampler = test_sampler2, batch_size=BATCH_SIZE)

In [ ]:
score_metrics2, t_predictions2 = predict(model, test_dataloader2)

In [ ]:
score_metrics2

# Final Test
With random clain taken from internet

In [ ]:
#Test Set is the covid Dataset
#Replace this path with dataset that you want to test the model with
ctest_path = "/home/8vijayak/covid19-hackaton/dataset/covid_test.tsv" 
test_COVID_df = pd.read_csv(ctest_path, sep='\t')
test_COVID_df.columns = ["Statement", "Justification", "Label"]
test_COVID_df.dropna()
print("Length of Test Set:" + str(len(test_COVID_df)))
test_COVID_df.head()

In [ ]:
covid_text_test = list(test_COVID_df['Statement'])
covid_justification_test = list(test_COVID_df['Justification'])
covid_labels_test = list(test_COVID_df['Label'])

In [ ]:
ctext_tokens_test = tokenizer.batch_encode_plus(
    covid_text_test,
    max_length = 100,
    padding=True,
    truncation=True
)

cjustification_tokens_test = tokenizer.batch_encode_plus(
    covid_justification_test,
    max_length = 400,
    padding=True,
    truncation=True
)

In [ ]:
ctext_seq_test = torch.tensor(ctext_tokens_test['input_ids'])
cjustification_seq_test = torch.tensor(cjustification_tokens_test['input_ids'])
ctest_seq = torch.cat((ctext_seq_test, cjustification_seq_test), 1)

ctext_mask_test = torch.tensor(ctext_tokens_test['attention_mask'])
cjustification_mask_test = torch.tensor(cjustification_tokens_test['attention_mask'])
ctest_mask = torch.cat((ctext_mask_test, cjustification_mask_test), 1)

ctest_y = torch.tensor(covid_labels_test)

In [ ]:
ctest_data = TensorDataset(ctest_seq, ctest_mask, ctest_y)
ctest_sampler = SequentialSampler(ctest_data)
ctest_dataloader = DataLoader(ctest_data, sampler = ctest_sampler, batch_size=BATCH_SIZE)

In [ ]:
covid_score_metrics, covid_t_predictions2 = predict(model, ctest_dataloader)

In [ ]:
print("Mixed Model: Test Set: Covid:")
print(covid_score_metrics)

### Plots of Training Losses

In [ ]:
trainprogressfile = "TrainedModels/ds_weights_train_progress_ep-30_lr-1e05.tsv"

In [ ]:
df_trainProgress = pd.read_csv(trainprogressfile, sep='\t')

In [ ]:
df_trainProgress

In [ ]:
from plotly.graph_objs import *
import plotly.graph_objects as go

layout = Layout(
    title='Training Progress: Cross Entropy Loss & Accuracy on Training Dataset',
    xaxis = go.layout.XAxis(title = go.layout.xaxis.Title(text='Number of Epochs')),
    yaxis = go.layout.YAxis(title = go.layout.yaxis.Title(text='Cross Entropy Loss while Training')),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')

fig = go.Figure(layout=layout)
fig.add_scatter(x=df_trainProgress['epoch'], y=df_trainProgress['train_loss'],mode='lines', line_color='blueviolet', name = "Training Loss")
fig.add_scatter(x=df_trainProgress['epoch'], y=df_trainProgress['train_accuracy'],mode='lines', line_color='teal', name = "Training Accuracy")

fig.show()

In [ ]:
from plotly.graph_objs import *
import plotly.graph_objects as go

layout = Layout(
    title='Training Progress: Cross Entropy Loss & Accuracy on Validation Dataset',
    xaxis = go.layout.XAxis(title = go.layout.xaxis.Title(text='Number of Epochs')),
    yaxis = go.layout.YAxis(title = go.layout.yaxis.Title(text='Cross Entropy Loss while Training')),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')

fig = go.Figure(layout=layout)
fig.add_scatter(x=df_trainProgress['epoch'], y=df_trainProgress['eval_loss'],mode='lines', line_color='blueviolet', name = "Evaluation Loss")
fig.add_scatter(x=df_trainProgress['epoch'], y=df_trainProgress['eval_accuracy'],mode='lines', line_color='teal', name = "Evaluation Accuracy")

fig.show()